# Credit mining

A Job can generate new credits with time, by bonding Keep3r Liquidity Pool tokens kLP to it. Liquidities will be handled by the Keep3r Liquidity Pools (`keeep3r.pool`).
Once kLPs are added to a job with addLiquidityToJob, the job starts immediately to mint new KP3R credits, that can be collectable only by the keepers, in reward for working the job. The credit minting system requires no further action from the job owner (`provider`).

### Setup
Run first `00-setup.ipynb` and select its kernel

In [4]:
// create job and add liquidity

next(async()=>{
    job = await(await ethers.getContractFactory('JobForTest')).connect(provider).deploy(keep3r.v2.address)

    await keep3r.v2.connect(provider).addJob(job.address)
    await keep3r.pool.connect(provider).approve(keep3r.v2.address, klpBalance)
    await keep3r.v2.connect(provider).addLiquidityToJob(job.address, keep3r.pool.address, klpBalance)
})

In [5]:
// charts configuration

next(async() =>{
    await $.resetRecording()
    $.resetTraces()

    $.setPeriodTrace(432000)
    
    $.addViewTrace(keep3r.v2, 'jobLiquidityCredits', [job.address])
    $.addViewTrace(keep3r.v2, 'totalJobCredits', [job.address])
    $.addViewTrace(keep3r.v2, 'jobPeriodCredits', [job.address]) 
//     TODO: add multiple arguments to view
    $.addViewTrace(keep3r.v1, 'bonds', [keep3r.v1.address, keep3r.keeper.address]) 
    $.addEventTrace(keep3r.v2.web3, 'LiquidityCreditsReward', '_rewardedAt')
    $.addEventTrace(keep3r.v2.web3, 'LiquidityAddition')
    $.addEventTrace(keep3r.v2.web3, 'KeeperWork')
    $.addEventTrace(uniV3Pool.web3, 'Swap')    
})

### Reward Periods

To handle KP3R credits minting and quoting, Keep3r introduces reward periods, in which KP3R quote remains stable for each pair, and gas-efficiently processed. These quotes are used within the protocol to mint credits and reward keepers.

The underlying KP3R of the liquidity provided, should generate the same amount of KP3R every inflationPeriod, thereby minting the proportional amount each rewardPeriod as KP3R credits for the job.

In [6]:
// liquidity added in 00-setup

next(async()=>{
    rewardPeriodTime = (await keep3r.v2.rewardPeriodTime()).toNumber()
    inflationPeriod = (await keep3r.v2.inflationPeriod()).toNumber()
    
    console.log('KP3R spent', bnToNumber(kp3rSpent))
    console.log('WETH spent', bnToNumber(wethSpent))
    console.log('kLP minted', bnToNumber(klpBalance))
    console.log('reward period',  rewardPeriodTime / (3600 * 24), 'days')
    console.log('inflation period', (await keep3r.v2.inflationPeriod()).toNumber() / (3600 * 24), 'days')
    console.log('KP3R minted by reward period', bnToNumber(await keep3r.v2.jobPeriodCredits(job.address)))
    console.log('KP3R minted by inflation period', bnToNumber((await keep3r.v2.jobPeriodCredits(job.address)).mul(inflationPeriod).div(rewardPeriodTime)))
})

KP3R spent 9.99
WETH spent 1.3
kLP minted 3.6
reward period 5 days
inflation period 34 days
KP3R minted by reward period 1.48
KP3R minted by inflation period 10.09


To determine the value of a certain liquidity, Keep3r uses a TWAP calculation to get the average quote of a pair in the last completed epoch. The same calculation is applied to quote rewards for keepers (that spend gas in ETH and receive KP3R rewards), using a predefined KP3R/WETH pool as an oracle.

- A job will mint the result of quoteLiquidity every rewardPeriodTime 
- Keep3r will store and use the average quote for the last epochfor each given liquidity
- Remaining credits will be updated to current quotes each time a `rewardPeriod` starts
- Credits older than previous `rewardPeriod` will no longer be updateable and will expire

In [7]:
// credit mining without working

next(async()=>{
    
    await $.sleepAndRecord(
        3 * rewardPeriodTime,
        $.time(4,'hours')
    )
    
    await $.draw()
})

##### Reward periods

KP3R quote, in relation with each liquidity, are updated every period, meaning than during each period, it remains stable for credit minting and for payments

In [8]:
// twap calculation: 1.0001^(tickDifference/timeDifference)

uniQuote = async()=>{   
    tickTime = rewardPeriodTime
    uniResponse = await uniV3Pool.observe([0,tickTime])
    quote = 1.0001**(((uniResponse[0][1]).sub(uniResponse[0][0])).div(tickTime))
    console.log('tickQuote', quote)
}

[AsyncFunction: uniQuote]

In [9]:
// credit mining with twap change

next(async()=>{    
    await $.resetRecording()
    
    await $.sleepAndExecute(
        3 * rewardPeriodTime,
        $.time(12,'hours'),
        // make a big swap in uniswapV3 pool to alter quote
        [{
            run: async()=>{
                    await common.makeASwap(
                            provider, 
                            keep3r.v1.address, 
                            constants.WETH_ADDRESS, 
                            provider.address, 
                            1000, 
                            toUnit(300))
                    await uniQuote()
            },
            every: $.time(2,'days')
         }]
    )
    
    await $.draw()
})

tickQuote 7.673693966732986
tickQuote 7.800573774683414
tickQuote 8.060661716389532
tickQuote 8.349435168414958
tickQuote 8.667600748322114


In [10]:
// balanced work

next(async()=>{
    await $.sleep(rewardPeriodTime)
    await $.resetRecording()
        
    await $.sleepAndExecute(
        4 * rewardPeriodTime,
        $.time(4,'hours'),
        [{run: async()=>{
            await job.connect(keep3r.keeper).workHard(2)
        },
        every: $.time(36,'hours')}]
    )
    
    await $.draw()
})

In [11]:
// balanced work with twap change

next(async()=>{
    await $.sleep(rewardPeriodTime)
    await $.resetRecording()
        
    await $.sleepAndExecute(
        4 * rewardPeriodTime,
        $.time(4,'hours'),
        [{
            run: async()=>{
                await job.connect(keep3r.keeper).workHard(2)
            },
            every: $.time(36,'hours'),
        },
        {
            run: async()=>{
                await common.makeASwap(provider, keep3r.v1.address, constants.WETH_ADDRESS, provider.address, 1000, toUnit(300))
            },
            every: $.time(5,'days')
        }]
    )
    
    await $.draw()
})

In a normal case scenario, a job should be rewarded once every rewardPeriodStart and burn all remaining credits from last epoch. When jobs are running low on credits, they will be rewarded before the next rewardPeriod starts, and those credits will be minted with the previous epoch quote, so when next rewardPeriod starts, and liquidity quotes are updated, the job will have its remaining KP3R credits updated to the new quotes.

In [12]:
// intense work

next(async()=>{
    await $.sleep(rewardPeriodTime)
    await $.resetRecording()
    block = await $.block()
    await advanceTimeAndBlock(rewardPeriodTime - (block.timestamp % rewardPeriodTime))
    
    await $.sleepAndExecute(
        2 * rewardPeriodTime,
        $.time(4,'hours'),
        [{run: async()=>{
            await job.connect(keep3r.keeper).workHard(3)
        },
        every: $.time(36,'hours')}]
    )
 
    await $.draw()
})

On a deficitary job scenario, when the job is spending more KP3R in a period than it is rewarded, the job will be able to keep on paying keepers, but the minting period starts to shrink, having to mint more frequently, and rewarding extra credits for the keeper, every time he has to reward the job to get his payment.

Each time, the reward period will be shorter, as the job is not being rewarded the full amount for a period, but only the proportional relation of the time that passed since last reward, and the rewardPeriod.

Ultimately, when the job has not enough `totalJobCredits` to reward the keeper for working the job (and some extra to pay the keeper for rewarding the credits), the transaction will revert with `InsufficientFunds`.

##### Maximum credit spending

Since jobs are able to be worked as long as `totalJobCredits > payment`, it is possible for the jobs to have a spending bigger than their current credits, only if they have minted so far enough to make the payment. If a job has been minting credits (for a maximum of a `rewardPeriodTime`), and preserves its current credits, it can spend both on a single payment, greater than `jobPeriodCredits` and no greater than `2 * jobPeriodCredits`.

In [13]:
// maximum credit spending

next(async()=>{
    await $.resetRecording()
    block = await $.block()
    await $.sleep(rewardPeriodTime)
    await advanceTimeAndBlock(rewardPeriodTime - (block.timestamp % rewardPeriodTime))
    
    await $.sleepAndExecute(
        2 * Math.floor(1.9 * rewardPeriodTime),
        $.time(4,'hours'),
        [{run: async()=>{
            await job.connect(keep3r.keeper).workHard(10)
        },
        every: Math.floor(1.8 * rewardPeriodTime)}]
    )
 
    await $.sleepAndRecord(rewardPeriodTime, $.time(4,'hours'))
    
    await $.draw()
})

##### Removing liquidity

When a liquidity is removed, Keep3r will recalculate the corresponding `jobPeriodCredits` and reduce current credits proportionally to the impact.

In [14]:
// unbonding liquidity

next(async()=>{
    await $.resetRecording()
    block = await $.block()
    await advanceTimeAndBlock(rewardPeriodTime - (block.timestamp % rewardPeriodTime))
    
    await $.sleepAndExecute(
        2 * rewardPeriodTime,
        $.time(4,'hours'),
        [{run: async()=>{
            await keep3r.v2.connect(provider).unbondLiquidityFromJob(job.address, keep3r.pool.address, toUnit(1))
        },
        every: rewardPeriodTime}]
    )
 
    await $.sleepAndRecord(rewardPeriodTime, $.time(4,'hours'))
    
    await $.draw()
})